In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import pandas as pd
tf.config.set_soft_device_placement(False)
tf.debugging.set_log_device_placement(True)
import numpy as np
from ampligraph.datasets import load_fb15k_237, load_yago3_10
from ampligraph.evaluation.protocol import create_mappings, to_idx

from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score
import time
print(tf.__version__)
assert(tf.__version__.startswith('2.3'))

from ampligraph.datasets import load_fb15k_237, load_fb13, load_fb15k, load_wn11, load_wn18, load_wn18rr, load_yago3_10
from ampligraph.latent_features import ScoringBasedEmbeddingModel

2.3.0


In [2]:
#loss function
def nll(scores_pred, eta):
    scores_neg = scores_pred[1]
    scores_pos = scores_pred[0]

    scores_neg_reshaped = tf.reshape(scores_neg, [eta, tf.shape(scores_pos)[0]])
    neg_exp = tf.exp(scores_neg_reshaped)
    pos_exp = tf.exp(scores_pos)
    softmax_score = pos_exp / (tf.reduce_sum(neg_exp, axis=0) + pos_exp)

    loss = -tf.reduce_sum(tf.math.log(softmax_score))
    return loss

# Train/eval without partition

In [3]:
model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     max_ent_size=14505, 
                                     max_rel_size=237,
                                     scoring_type='TransE')

model.compile(optimizer='adam', loss=nll)

from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./chkpt1', monitor='loss', verbose=1, save_best_only=True, mode='min')

model.fit('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
         batch_size=1000,
         epochs=10, callbacks=[checkpoint])



Epoch 1/10
    272/Unknown - 5s 17ms/step - loss: 1499.1533
Epoch 00001: loss improved from inf to 1497.92505, saving model to ./chkpt1
273/273 [==============================] - 5s 17ms/step - loss: 1497.9250
Epoch 2/10
272/273 [============================>.] - ETA: 0s - loss: 1217.3185
Epoch 00002: loss improved from 1497.92505 to 1216.53357, saving model to ./chkpt1
273/273 [==============================] - 5s 17ms/step - loss: 1216.5336
Epoch 3/10
272/273 [============================>.] - ETA: 0s - loss: 1031.2305
Epoch 00003: loss improved from 1216.53357 to 1030.67896, saving model to ./chkpt1
273/273 [==============================] - 5s 17ms/step - loss: 1030.6790
Epoch 4/10
271/273 [============================>.] - ETA: 0s - loss: 903.4239
Epoch 00004: loss improved from 1030.67896 to 902.64667, saving model to ./chkpt1
273/273 [==============================] - 5s 17ms/step - loss: 902.6467
Epoch 5/10
272/273 [============================>.] - ETA: 0s - loss: 810.1034
Epo

In [4]:

start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt')
end = time.time()
mr_score(ranks), mrr_score(ranks)

Invalid Key encountered in  ['/m/016zwt' '/location/location/contains' '/m/08hbxv']
Invalid Key encountered in  ['/m/05hyf' '/film/film_subject/films' '/m/02yvct']
Invalid Key encountered in  ['/m/0qb7t'
 '/user/tsegaran/random/taxonomy_subject/entry./user/tsegaran/random/taxonomy_entry/taxonomy'
 '/m/04n6k']
Invalid Key encountered in  ['/m/027752' '/common/topic/webpage./common/webpage/category' '/m/08mbj5d']
Invalid Key encountered in  ['/m/03f_jk' '/common/topic/webpage./common/webpage/category' '/m/08mbj5d']
Invalid Key encountered in  ['/m/027qb1'
 '/business/job_title/people_with_this_title./business/employment_tenure/company'
 '/m/03z19']
Invalid Key encountered in  ['/m/01x8f6' '/common/topic/webpage./common/webpage/category' '/m/08mbj5d']
Invalid Key encountered in  ['/m/025x7g_'
 '/user/tsegaran/random/taxonomy_subject/entry./user/tsegaran/random/taxonomy_entry/taxonomy'
 '/m/04n6k']
Invalid Key encountered in  ['/m/080v2' '/common/topic/webpage./common/webpage/category' '/m

(437.08630485893417, 0.09560395825092462)

# Training/eval with partition

In [5]:
from ampligraph.datasets import DummyBackend, SQLiteAdapter
from ampligraph.datasets import GraphDataLoader
from ampligraph.datasets.graph_partitioner import PARTITION_ALGO_REGISTRY
dataset_loader = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt', 
                                 backend=SQLiteAdapter,
                                    batch_size=1000, 
                                    dataset_type='train', 
                                    epochs=10,
                                    use_indexer=True)


partitioner = PARTITION_ALGO_REGISTRY.get('Bucket')(dataset_loader, k=3)

_split: memory before: 0.0Bytes, after: 13.0MB, consumed: 13.0MB; exec time: 208.99s


In [6]:
model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     max_ent_size=9437, 
                                     max_rel_size=237,
                                     scoring_type='TransE')

model.compile(optimizer='adam', loss=nll)

from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./chkpt1', monitor='loss', verbose=1, save_best_only=True, mode='min')

model.fit(partitioner,
         batch_size=1000, use_partitioning=True,
         epochs=10, callbacks=[checkpoint])

Epoch 1/10
    277/Unknown - 10s 36ms/step - loss: 1674.6713
Epoch 00001: loss improved from inf to 1674.67126, saving model to ./chkpt1
277/277 [==============================] - 11s 39ms/step - loss: 1674.6713
Epoch 2/10
276/277 [============================>.] - ETA: 0s - loss: 1552.4003
Epoch 00002: loss improved from 1674.67126 to 1549.75720, saving model to ./chkpt1
277/277 [==============================] - 9s 31ms/step - loss: 1549.7572
Epoch 3/10
276/277 [============================>.] - ETA: 0s - loss: 1434.9354
Epoch 00003: loss improved from 1549.75720 to 1433.30750, saving model to ./chkpt1
277/277 [==============================] - 9s 31ms/step - loss: 1433.3075
Epoch 4/10
276/277 [============================>.] - ETA: 0s - loss: 1332.2727
Epoch 00004: loss improved from 1433.30750 to 1331.13989, saving model to ./chkpt1
277/277 [==============================] - 9s 31ms/step - loss: 1331.1399
Epoch 5/10
276/277 [============================>.] - ETA: 0s - loss: 1244.45

In [7]:

start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=1000)
end = time.time()
mr_score(ranks), mrr_score(ranks)

Invalid Key encountered in  ['/m/016zwt' '/location/location/contains' '/m/08hbxv']
Invalid Key encountered in  ['/m/05hyf' '/film/film_subject/films' '/m/02yvct']
Invalid Key encountered in  ['/m/0qb7t'
 '/user/tsegaran/random/taxonomy_subject/entry./user/tsegaran/random/taxonomy_entry/taxonomy'
 '/m/04n6k']
Invalid Key encountered in  ['/m/027752' '/common/topic/webpage./common/webpage/category' '/m/08mbj5d']
Invalid Key encountered in  ['/m/03f_jk' '/common/topic/webpage./common/webpage/category' '/m/08mbj5d']
Invalid Key encountered in  ['/m/027qb1'
 '/business/job_title/people_with_this_title./business/employment_tenure/company'
 '/m/03z19']
Invalid Key encountered in  ['/m/01x8f6' '/common/topic/webpage./common/webpage/category' '/m/08mbj5d']
Invalid Key encountered in  ['/m/025x7g_'
 '/user/tsegaran/random/taxonomy_subject/entry./user/tsegaran/random/taxonomy_entry/taxonomy'
 '/m/04n6k']
Invalid Key encountered in  ['/m/080v2' '/common/topic/webpage./common/webpage/category' '/m

(1263.682425, 0.08404883215972675)